In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [2]:
from pathlib import Path

print(Path.cwd())  # /home/skovorodkin/stack

/workspaces/src/trader_buddy/jupyter-notebooks


In [2]:
import pandas as pd
from datetime import datetime

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

In [5]:
import os
import sys
sys.path.append("../projects/gain_tracker")
from gain_tracker.position import ClosedPosition

In [6]:
query_job = client.query("SELECT * FROM `main-street-labs.gain_tracker.closed_positions`")
closed_positions = query_job.result().to_dataframe()

In [126]:
len(closed_positions)

56

In [7]:
closed_positions.drop_duplicates(subset=["POSITION_LOT_ID", "QUANTITY", "DATE_CLOSED"], inplace=True)

In [8]:
closed_positions.dtypes

SYMBOL_DESCRIPTION                 object
QUANTITY                          float64
DATE_ACQUIRED                      dbdate
PRICE_PAID                        float64
ACCOUNT_ID_KEY                     object
POSITION_ID                         Int64
POSITION_LOT_ID                     Int64
ORIGINAL_QTY                      float64
TIMESTAMP             datetime64[us, UTC]
DATE_CLOSED                        dbdate
TRANSACTION_FEE                   float64
TRANSACTION_ID                      Int64
MARKET_VALUE                      float64
dtype: object

In [26]:
closed_gains_df.dtypes

SYMBOL_DESCRIPTION                  object
QUANTITY                           float64
DATE_ACQUIRED                       dbdate
PRICE_PAID                         float64
ACCOUNT_ID_KEY                      object
POSITION_ID                          Int64
POSITION_LOT_ID                      Int64
ORIGINAL_QTY                       float64
TIMESTAMP              datetime64[us, UTC]
DATE_CLOSED                         dbdate
TRANSACTION_FEE                    float64
TRANSACTION_ID                       Int64
MARKET_VALUE                       float64
percent_price_gain                  object
gain                                object
percent_gain                        object
annualized_pct_gain                 object
days_held                          float64
dtype: object

In [35]:
positions = closed_positions.apply(
    lambda r: ClosedPosition(
        r["POSITION_LOT_ID"],
        r["ACCOUNT_ID_KEY"],
        r["SYMBOL_DESCRIPTION"],
        r["PRICE_PAID"],
        r["DATE_ACQUIRED"],
        r["QUANTITY"],
        r["ORIGINAL_QTY"],
        r["MARKET_VALUE"],
        transaction_id = r["TRANSACTION_ID"],
        transaction_fee = r["TRANSACTION_FEE"],
        date_closed=r["DATE_CLOSED"]), axis=1)

In [154]:
one_p = positions.values[0]

In [155]:
one_p.market_price()

18.99833333333333

In [ ]:
gmetrics = positions.apply(lambda p: p.compute_gains())

gm_df = pd.DataFrame(gmetrics.values.tolist())

closed_gains_df = pd.concat([closed_positions.reset_index(drop=True),gm_df],axis=1)

In [37]:
closed_gains_df.tail()

,SYMBOL_DESCRIPTION,QUANTITY,DATE_ACQUIRED,PRICE_PAID,ACCOUNT_ID_KEY,POSITION_ID,POSITION_LOT_ID,ORIGINAL_QTY,TIMESTAMP,DATE_CLOSED,TRANSACTION_FEE,TRANSACTION_ID,MARKET_VALUE,percent_price_gain,gain,percent_gain,annualized_pct_gain,days_held
49,DQ,10.0,2024-08-29,14.50,SXPtmpM7IZIpymUjhjI-zg,372865585900,2699858515103,10.0,2024-09-13 00:00:00+00:00,2024-09-13,0.0,24257201074125,161.39,0.11303448276,16.3900000002,0.11303448276,12.5428396917,15
50,NIO,30.0,2024-06-10,4.80,SXPtmpM7IZIpymUjhjI-zg,356289802900,2557349775103,30.0,2024-09-13 00:00:00+00:00,2024-09-13,0.0,24257200235909,158.99,0.10409722222,14.9899999997,0.104097222220,0.46298084520,95
51,HUBS,2.0,2024-08-20,490.00,SXPtmpM7IZIpymUjhjI-zg,396827605900,2685514461103,2.0,2024-09-13 00:00:00+00:00,2024-09-13,0.0,24257200325971,999.97,0.02037755102,19.9699999996,0.02037755102,0.35905995374,24
52,CRWD,1.0,2024-07-26,255.00,E2ayfoKSzH1fFZs-7xV48A,391087961900,2632002334103,1.0,2024-09-16 00:00:00+00:00,2024-09-16,0.0,24260201502940,269.99,0.05878431373,14.9900000012,0.0587843137302,0.49323979848,52
53,IEFA,20.0,2016-06-01,54.11,lP_HmzzVyiLFftb7DXaEUw,26413605005,20702960006,20.0,2024-09-16 00:00:00+00:00,2024-09-16,0.0,24260201374097,1527.95,0.41189244132,445.749999997,0.411892441320,0.04244071730,3029


In [38]:
job = client.load_table_from_dataframe(
    closed_gains_df.set_index('POSITION_LOT_ID'), "main-street-labs.gain_tracker.closed_positions"
)

# set datatypes

In [144]:
empty_ts = pd.DataFrame(
    {'account_id': pd.Series(dtype='str'),
    'transaction_type': pd.Series(dtype='str'),
    'transaction_date': pd.Series(dtype='datetime64[ns]')})

In [10]:
empty_ts.dtypes

account_id                  object
transaction_type            object
transaction_date    datetime64[ns]
dtype: object

# dataframe manip

In [23]:
x = pd.DataFrame(data={"a": [1,2, 3], "b": ["f", "g", None], "c": [5,66,1]}).set_index("a")

In [46]:
(~x.isna()).all(axis=1)

a
1     True
2     True
3    False
dtype: bool

In [47]:
x.loc[

,b,c
a,,
1,f,5
2,g,66
3,None,1


In [48]:
y = x.head(2).copy()
# x.loc[2, "b"] = "ff"
# x.loc[5] = pd.Series({"b": "t"})

# yfinance

In [29]:
import yfinance as yf

In [2]:
sp500 = yf.Ticker("IVV")

In [4]:
sp500.fast_info["timezone"]

'America/New_York'

In [6]:
sp500.info['navPrice']

542.5931

In [9]:
sp500.info['firstTradeDateEpochUtc']

958743000

In [7]:
sp500.fast_info["lastPrice"]

543.6699829101562

In [6]:
sp500.info["shortName"]

'iShares Core S&P 500 ETF'

In [10]:
sp500.info["uuid"]

'17732e4a-0401-33ba-977d-a9848ab99a1e'

In [11]:
sp500.info["exchange"]

'PCX'

In [12]:
sp500.info["open"]

489.94

In [13]:
sp500.info["bid"]

489.62

In [8]:
sp500.info["previousClose"]

542.53

In [17]:
sp500.info["volume"]

6635596

In [10]:
list(sp500.info)

['phone',
 'longBusinessSummary',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'trailingPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bidSize',
 'askSize',
 'yield',
 'totalAssets',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'trailingAnnualDividendRate',
 'trailingAnnualDividendYield',
 'navPrice',
 'currency',
 'category',
 'ytdReturn',
 'beta3Year',
 'fundFamily',
 'fundInceptionDate',
 'legalType',
 'threeYearAverageReturn',
 'fiveYearAverageReturn',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'trailingPegRatio']

In [18]:
# get historical market data
# hist = sp500.history(period="1mo")
hist = sp500.history(period="1day")

In [24]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2024-01-25 00:00:00-05:00,489.940002,490.619995,487.839996,490.329987,6615725,0.0,0.0,0.0


In [111]:
# co = yf.Ticker("TTEK")
# co = yf.Ticker("C")
co = yf.Ticker("MSFT")

In [112]:
# doesn't exist for TTEK
co.sustainability

,esgScores
adult,False
alcoholic,False
animalTesting,False
catholic,False
coal,False
controversialWeapons,False
environmentPercentile,None
environmentScore,1.46
esgPerformance,UNDER_PERF
furLeather,False


In [44]:
co.institutional_holders
# co.earnings_estimate

,Date Reported,Holder,pctHeld,Shares,Value
0,2023-06-30,Vanguard Group Inc,0.0863,339103490,5130635803
1,2023-06-30,Blackrock Inc.,0.0710,279184561,4224062407
2,2023-06-30,State Street Corporation,0.0430,168856962,2554805835
3,2023-06-30,"Newport Trust Company, LLC",0.0387,152155847,2302117965
4,2023-06-30,"Charles Schwab Investment Management, Inc.",0.0216,84734138,1282027507
5,2023-06-30,"Geode Capital Management, LLC",0.0184,72311048,1094066156
6,2023-06-30,"Fisher Asset Management, LLC",0.0154,60733261,918894238
7,2023-06-30,Morgan Stanley,0.0123,48518047,734078051
8,2023-06-30,Northern Trust Corporation,0.0093,36716417,555519389
9,2022-12-31,Norges Bank Investment Management,0.0092,36047929,419237414


In [7]:
revenue = 4522550

In [103]:
years = co.income_stmt.loc["Total Revenue"].to_frame().apply(lambda r: r.name.year, axis=1)

In [62]:
nx = yf.Ticker("2z")

In [72]:
nx.info.get("shortName")

In [108]:
co.info["shortName"]

'Citigroup, Inc.'

In [104]:
revs = co.income_stmt.loc["Total Revenue"]

In [105]:
revs

2023-12-31    1450871000.0
2022-12-31    1261868000.0
2021-12-31    1258846000.0
2020-12-31    1124391000.0
Name: Total Revenue, dtype: object

In [11]:
co.income_stmt.loc["Total Revenue", '2023-09-30'] == revenue*1000

True

In [12]:
co = yf.Ticker("600383")

In [15]:
co.info

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/600383?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=600383&crumb=M4Dp01e0VzE


{'trailingPegRatio': None}